# Simple Backup Script for NX-OS
Requirements:
 - Must have SSH key generated on switch, and public key copied to backup server
 - Enable NXAPI and NXAPI-Sandbox on N7k
 
Program Flow: 
 - Script runs as a cron job once daily on backup server.
 - Script connects to switch via NXAPI.
 - Script issues command: copy startup-config sftp://10.18.188.188/home/bkup/n7k/[name]-[date]
   - N7k copies startup-config to the backup server

On Switch: 
 - conf t
 - username admin keypair generate rsa
 - show username admin keypair

In [1]:
import requests
import json
import sys
from datetime import date

In [2]:
## Variables
today = str(date.today())

In [3]:
switch_ip = "10.18.188.137"
username = "admin"
password = ""

In [4]:
from requests.auth import HTTPBasicAuth
auth = HTTPBasicAuth(username, password)

In [5]:
auth_url = "http://" + switch_ip + "/ins" 

In [6]:
print(auth_url)

http://10.18.188.137/ins


In [7]:
print(auth)

In [8]:
show_config_payload = {
  "ins_api": {
    "version": "1.2",
    "type": "cli_show",
    "chunk": "0",
    "sid": "1",
    "input": "show running-config",
    "output_format": "json"
  }
}

In [22]:
bkup_host = "10.18.188.188"
bkup_user = "bkup"
proto_schema = "sftp"
bkup_device = "labcore01"
bkup_path = "/home/bkup/n7k/"
bkup_source_file = "startup-config"
bkup_dest_file = bkup_source_file + "_" + bkup_device + "_" + today + "-TESTING "
default_vrf = "vrf management"
input_cmd = "copy " + bkup_source_file + " " + proto_schema + "://" + bkup_user + "@" + bkup_host + bkup_path + bkup_dest_file + default_vrf

In [23]:
print(input_cmd)

copy startup-config sftp://bkup@10.18.188.188/home/bkup/n7k/startup-config_labcore01_2019-07-02-TESTING vrf management


In [24]:
bkup_payload = {
  "ins_api": {
    "version": "1.2",
    "type": "cli_conf",
    "chunk": "0",
    "sid": "1",
    "input": input_cmd,
    "output_format": "json"
  }
}

In [25]:
#r = requests.post(auth_url, json=auth_data, verify=False)
r = requests.post(auth_url, auth=auth, json=bkup_payload)

In [26]:
r.status_code

200

In [27]:
r.headers

{'Server': 'nginx/1.4.5', 'Date': 'Tue, 02 Jul 2019 20:22:49 GMT', 'Content-Type': 'text/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'nxapi_auth=admin:156209956203837432'}

In [21]:
#r.json
xml = r.content

In [22]:
r_json = r.json()

In [23]:
r_json

{'ins_api': {'sid': 'eoc',
  'type': 'cli_conf',
  'version': '1.2',
  'outputs': {'output': {'code': '500',
    'msg': 'Backend processing error',
    'input': 'copy startup-config sftp://bkup@10.18.188.188/home/bkup/n7k/startup-config-2019-06-26'}}}}